Here I will construct a CNN within tensorflow to identify the type of flower, won't actually be running this code here though becuase it requires a GPU, so that'll happen on a remote server but let's get the basics down here!


https://www.kaggle.com/alxmamaev/flowers-recognition

Should now only read in jpegs

Running cnn now, building/fixing architecture

In [1]:
%matplotlib inline


import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import os

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Part I
------
Making the dataset

Here I'm making the arrays of each flower type, defining their label, and then making a function to find the max height and width.

Not sure how to find the "nominal" height and width, but the code to find it is in here provided you know what you're looking for (I don't). 

In [ ]:
cd flowers/

In [ ]:
ls

In [2]:
#Making my daisy set in here

X_daisies = []
y_daisies = []
for file in os.listdir('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/daisy'):
    if file.endswith(".jpg"):
        img = cv2.imread('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/daisy/' + file)
        X_daisies.append(img)
        y_daisies.append([1,0,0,0,0])

        
        
#Now for dandelions
X_dandelions = []
y_dandelions = []
for file in os.listdir('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/dandelion'):
    #print(filename)
    if file.endswith(".jpg"):
        img = cv2.imread('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/dandelion/' + file)
        X_dandelions.append(img)
        y_dandelions.append([0,1,0,0,0])

#Now for roses
X_roses = []
y_roses = []
for file in os.listdir('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/rose'):
    #print(filename)
    if file.endswith(".jpg"):
        img = cv2.imread('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/rose/' + file)
        X_roses.append(img)
        y_roses.append([0,0,1,0,0])

#Now for sunflowers
X_sunflowers = []
y_sunflowers = []
for file in os.listdir('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/sunflower'):
    #print(filename)
    if file.endswith(".jpg"):
        img = cv2.imread('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/sunflower/' + file)
        X_sunflowers.append(img)
        y_sunflowers.append([0,0,0,1,0])

#Last but not least, tulips
X_tulips = []
y_tulips = []
for file in os.listdir('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/tulip'):
    #print(filename)
    if file.endswith(".jpg"):
        img = cv2.imread('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/tulip/' + file)
        X_tulips.append(img)
        y_tulips.append([0,0,0,0,1])

I've now created arrays for each of the flowers, now merging them all together...

In [3]:
#Merging all at once. 4236 flowers in all. 

X_args = (X_daisies,X_dandelions,X_roses,X_sunflowers,X_tulips)
y_args = (y_daisies,y_dandelions,y_roses,y_sunflowers,y_tulips)
X_flowers = np.concatenate(X_args)
y_flowers = np.concatenate(y_args)



After running the block below we can identify shitty data. Delete here. 
Done!

In [ ]:
maxone = 0
#Don't make this for just daisies, do it for all flowers and then you can make the black space
for i in range(len(X_flowers)):
    flower = X_flowers[i]
    height = np.shape(flower)[1] #*np.shape(daisy)[1]
    #Some er
    #print(i)
    if height>maxone:
        print(i)
        print("hmax  = " +str(height))
        maxone = height


In [ ]:
maxone = 0
#Don't make this for just daisies, do it for all flowers and then you can make the black space
for i in range(len(X_flowers)):
    flower = X_flowers[i]
    width = np.shape(flower)[0] #*np.shape(daisy)[1]
    #Some er
    #print(i)
    if width>maxone:
        print(i)
        print("wmax  = " +str(width))
        maxone = width


In [4]:
# Resize images
def resize_with_pad(img, img_size):
    height, width, _ = img.shape
    ratio = img_size / max(height, width)
   # print(ratio)
    if ratio < 1:
        img = cv2.resize(img, (int(ratio * width), int(ratio * height)))
    padding = ((img_size - img.shape[0], 0), (img_size - img.shape[1],0), (0,0))

    return np.pad(img, padding, 'constant')

def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

for i in range(0,len(X_flowers)):
    
    X_flowers[i] = resize_with_pad(X_flowers[i], 360)
    X_flowers[i] = rgb2gray(X_flowers[i])
   # X_flowers[i] = np.array(X_flowers[i]).reshape(1, 360,360,3) useless!
    #Extra line here reshapes to accept batch size as a dimension of accepted pics at once

In [5]:
np.shape(X_flowers[0])

(360, 360)

Not really sure what this error is, probs worth investigating, but it looks like the max height is 640. Keep in mind this data set isn't clean, there's like a python script inside it and otehr things that could explain this issue and will go away once removed. 

After making this data set the rest is using a CNN which is conveniently explained how to make below!


So basically the cells above convert this into an array, (1066, 1599, 3)  3 representing RGB vals, so that's on track with this flower recog software

Next up is to design the CNN... 

Pulling from Aymerican Dreams...




In [ ]:
np.shape(X_flowers[6])

In [6]:
#should convert vals to float 32, will get on that later.
# Training Parameters
learning_rate = 0.001
num_steps = 500
batch_size = 1
display_step = 10

# Network Parameters
num_input = np.shape(X_flowers[0])[0]*np.shape(X_flowers[0])[1]
num_classes = 5 #total flower classes, daisy, etc. 5 total 
dropout = 0.5 # Dropout, probability to keep units

# tf Graph input
#X = tf.placeholder(tf.float32, [None, num_input])
X = tf.placeholder(tf.float32,[None, 360*360]) #28 * 28 taille image 1 = 1pixel car noir et blanc "X" valeur
X_image = tf.reshape(X, [-1, 360, 360, 1])  #image of image reshaped

Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [7]:

# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    
    ###### Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 360,360,1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [8]:

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)


In [9]:
#Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [10]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = next_batch(batch_size,X_flowers,y_flowers)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    #print("Testing Accuracy:", \
    #    sess.run(accuracy, feed_dict={X: X_flowers,
                      #                Y: y_flowers}))
    

ValueError: Cannot feed value of shape (1, 360, 360) for Tensor 'Placeholder:0', which has shape '(?, 129600)'

In [ ]:

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 256 MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                      Y: mnist.test.labels[:256],
                                      keep_prob: 1.0}))

In [ ]:
129600

In [ ]:
np.shape(X_flowers[0])[0]*np.shape(X_flowers[0])[1]

In [ ]:
360*360

In [ ]:

def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

In [ ]:
rgb2gray(X_flowers[0])

In [ ]:
np.shape(rgb2gray(X_flowers[0]))

In [11]:

def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    # stride [1, x_movement, y_movement, 1]
    # Must have strides[0] = strides[3] = 1
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    # stride [1, x_movement, y_movement, 1]
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, 360*360])/255.   # 28x28
ys = tf.placeholder(tf.float32, [None, 5])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs, [-1, 360, 360, 1])
# print(x_image.shape)  # [n_samples, 28,28,1]

## conv1 layer ##
W_conv1 = weight_variable([5,5, 1,32]) # patch 5x5, in size 1, out size 32
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) # output size 28x28x32
h_pool1 = max_pool_2x2(h_conv1)                                         # output size 14x14x32

## conv2 layer ##
W_conv2 = weight_variable([5,5, 32, 64]) # patch 5x5, in size 32, out size 64
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) # output size 14x14x64
h_pool2 = max_pool_2x2(h_conv2)                                         # output size 7x7x64

## fc1 layer ##
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
# [n_samples, 7, 7, 64] ->> [n_samples, 7*7*64]
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## fc2 layer ##
W_fc2 = weight_variable([1024, 5])
b_fc2 = bias_variable([5])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


# the error between prediction and real data
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                              reduction_indices=[1]))       # loss
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

sess = tf.Session()
# important step
# tf.initialize_all_variables() no long valid from
# 2017-03-02 if using tensorflow >= 0.12
if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
    init = tf.initialize_all_variables()
else:
    init = tf.global_variables_initializer()
sess.run(init)
for i in range(1000):
    batch_xs, batch_ys = next_batch(batch_size,X_flowers,y_flowers)
    sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5})
    if i % 50 == 0:
        print(compute_accuracy(
            mnist.test.images[:1000], mnist.test.labels[:1000]))

ValueError: Cannot feed value of shape (1, 360, 360) for Tensor 'truediv:0', which has shape '(?, 129600)'